# Simple Chat

The most simple agent system

### Lang Graph

Agentensysteme sind Zustandsautomaten die "kompiliert" werden, und statisch sind

In [ ]:
from typing import Annotated, TypedDict

from IPython.display import Image, display
from langchain_openai import AzureChatOpenAI
from langgraph.graph import END, START, StateGraph
from langgraph.graph.message import add_messages

llm = AzureChatOpenAI(model="gpt4o",azure_deployment="ara-gpt4o", temperature=0.7)

class State(TypedDict):
    """The state of the agent."""
    messages: Annotated[list, add_messages]


def SimpleAgent(state: State):
    """The agent."""
    return {"messages": [llm.invoke(state["messages"])]}

# Baue Zustandsautomaten
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", SimpleAgent)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)
graph = graph_builder.compile()

display(Image(graph.get_graph().draw_mermaid_png()))




In [ ]:
def stream_graph_updates(user_input: str):
    """Send user input to the graph and print the responses."""
    for event in graph.stream({"messages": [{
            "role": "human",
            "content": user_input,
        }]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)


while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break

    stream_graph_updates(user_input)


### Schwarm

Keine Notwendigkeit zu wissen wie ein Agentensystem zur Laufzeit ausschaut.
Alles dynamisierbar, und manipulierbar

Agenten entscheiden wen sie als nächstes aufrufen, nicht das übergeordnete System!

In [ ]:
from schwarm.core.schwarm import Schwarm
from schwarm.models.agents.user_agent import UserAgent
from schwarm.models.types import Agent
from schwarm.provider.llm_provider import LLMConfig

chat_agent = Agent(name="chat_agent", configs=[LLMConfig(name="gpt-4o", streaming=True)])
user_agent = UserAgent(agent_to_pass_to=chat_agent, default_handoff_agent=True)

response = Schwarm().quickstart(user_agent)

### Autogen

### CrewAi